# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [1]:
# Load the https://www.residentadvisor.net/events page in your browser.

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [2]:
# Open the inspect element feature in your browser

## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [1]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
#Exploration; designing/testing function parts
response = requests.get("https://www.residentadvisor.net/events/us/losangeles")
soup = BeautifulSoup(response.content, 'html.parser')
event_listings = soup.find('div', id="event-listing")
entries = event_listings.findAll('li')
print(len(entries), entries[0])

83 <li><p class="eventDate date"><a href="/events.aspx?ai=23&amp;v=day&amp;mn=12&amp;yr=2019&amp;dy=26"><span>Thu, 26 Dec 2019 /</span></a></p></li>


In [2]:
#Successive exploration in function development
rows = []
for entry in entries:
    #Is it a date? If so, set current date.
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.head()

,0,1,2,3
0,"Conforce, A.Brehme, Andrew Kelley in-Store",Stellar Remnant,"Thu, 26 Dec 2019 /",9.0
1,Project Blowed 25th Anniversary Concert,Los Globos,"Thu, 26 Dec 2019 /",3.0
2,Club Surge Long Beach,Que Sera,"Thu, 26 Dec 2019 /",1.0
3,Watkins Live Soulful House Music,Harvelles,"Thu, 26 Dec 2019 /",1.0
4,Jupiter by Stealth Nights (POP UP),TBA - Los Angeles,"Thu, 26 Dec 2019 /",1.0


In [3]:
#Final function
def scrape_events(events_page_url):
    #Your code here
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        #Is it a date? If so, set current date.
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.head()
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

## Write a Function to Retrieve the URL for the Next Page

In [4]:
#Function development cell
soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']

'/events/us/losangeles/week/2020-01-02'

In [5]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    #Your code here
    return next_page_url

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [7]:
#Your code here
dfs = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/losangeles"
while total_rows <= 1000:
    print(total_rows)
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    print(cur_url)
    try:
        cur_url = next_page(cur_url)
    except:
        break
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:1000]
print(len(df))
df.head()

0
https://www.residentadvisor.net/events/us/losangeles
72
https://www.residentadvisor.net/events/us/losangeles/week/2020-01-02
144
https://www.residentadvisor.net/events/us/losangeles/week/2020-01-09
216
https://www.residentadvisor.net/events/us/losangeles/week/2020-01-16
288
https://www.residentadvisor.net/events/us/losangeles/week/2020-01-23
360
https://www.residentadvisor.net/events/us/losangeles/week/2020-01-30
432
https://www.residentadvisor.net/events/us/losangeles/week/2020-02-06
504
https://www.residentadvisor.net/events/us/losangeles/week/2020-02-13
576
https://www.residentadvisor.net/events/us/losangeles/week/2020-02-20
648
https://www.residentadvisor.net/events/us/losangeles/week/2020-02-27
720
https://www.residentadvisor.net/events/us/losangeles/week/2020-03-05
792
https://www.residentadvisor.net/events/us/losangeles/week/2020-03-12
864
https://www.residentadvisor.net/events/us/losangeles/week/2020-03-19
936
https://www.residentadvisor.net/events/us/losangeles/week/2020-03-

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,"Conforce, A.Brehme, Andrew Kelley in-Store",Stellar Remnant,"Thu, 26 Dec 2019 /",9.0
1,Project Blowed 25th Anniversary Concert,Los Globos,"Thu, 26 Dec 2019 /",3.0
2,Club Surge Long Beach,Que Sera,"Thu, 26 Dec 2019 /",1.0
3,Watkins Live Soulful House Music,Harvelles,"Thu, 26 Dec 2019 /",1.0
4,Jupiter by Stealth Nights (POP UP),TBA - Los Angeles,"Thu, 26 Dec 2019 /",1.0


## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!